In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import json

# 設定 Chrome 瀏覽器的選項
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized") # Chrome 瀏覽器在啟動時最大化視窗
options.add_argument("--incognito") # 無痕模式
options.add_argument("--disable-popup-blocking") # 停用 Chrome 的彈窗阻擋功能。

# 建立 Chrome 瀏覽器物件
driver = webdriver.Chrome(options=options)
driver.get("https://tw.buy.yahoo.com/")

In [ ]:
# 還沒處理刪除輸入值的部分
search_product = "皮夾"
driver.find_element(By.CSS_SELECTOR, "form input[type='search']").send_keys(search_product)

In [18]:
from selenium.webdriver.common.keys import Keys

input_box = driver.find_element(By.CSS_SELECTOR, "form input[type='search']")

input_box.send_keys(Keys.CONTROL, "a")
input_box.send_keys(Keys.DELETE)
input_box.send_keys(search_product)

In [8]:
driver.find_element(By.CSS_SELECTOR, "form button[type='submit']").click()

In [ ]:
newest = driver.find_elements(By.CSS_SELECTOR, ".SortBar_popup_1UvPW a")[0].get_attribute("href")
higher_rating = driver.find_elements(By.CSS_SELECTOR, ".SortBar_popup_1UvPW a")[1].get_attribute("href")
from_lowprice = driver.find_elements(By.CSS_SELECTOR, ".SortBar_sortBtns_RCto5 > a")[0].get_attribute("href")
from_highprice = driver.find_elements(By.CSS_SELECTOR, ".SortBar_sortBtns_RCto5 > a")[1].get_attribute("href")
most_hot_sale = driver.find_elements(By.CSS_SELECTOR, ".SortBar_sortBtns_RCto5 > a")[2].get_attribute("href")
method = {
    "最新上架" : newest,
    "評價" : higher_rating,
    "價格低到高" : from_lowprice,
    "價格高到低" : from_highprice,
    "近期熱銷" : most_hot_sale
}
driver.get(method["價格低到高"])

https://tw.buy.yahoo.com/search/product?p=%E7%9A%AE%E5%A4%BE&sort=-ptime
https://tw.buy.yahoo.com/search/product?p=%E7%9A%AE%E5%A4%BE&sort=-global_rating
https://tw.buy.yahoo.com/search/product?p=%E7%9A%AE%E5%A4%BE&sort=price
https://tw.buy.yahoo.com/search/product?p=%E7%9A%AE%E5%A4%BE&sort=-price
https://tw.buy.yahoo.com/search/product?p=%E7%9A%AE%E5%A4%BE&sort=-sales


In [127]:
driver.execute_script(f"window.scrollBy(0, 400);")

In [128]:
def scroll_down_slowly():

    last_height = driver.execute_script("return window.scrollY")
    page_height = driver.execute_script("return document.body.scrollHeight")

    while last_height < page_height:
        # 滾動一步
        driver.execute_script(f"window.scrollBy(0, 400);")

        # 等待 lazy-load 元素載入
        time.sleep(0.3)

        # 更新目前高度與頁面總高度(總高度會因為載入變高)
        new_height = driver.execute_script("return window.scrollY")
        page_height = driver.execute_script("return document.body.scrollHeight")

        # 沒繼續滾動就代表到底了
        if new_height == last_height:
            break

        last_height = new_height
scroll_down_slowly()

In [137]:
item_list = driver.find_elements(By.CSS_SELECTOR, "ul.gridList > a")
print(len(item_list))

0


In [134]:
def get_item_info():
    item_list = driver.find_elements(By.CSS_SELECTOR, "ul.gridList > a")
    for item in item_list:
        try:
            item_name = item.find_element(By.CSS_SELECTOR,".sc-1drl28c-4 > span").text
        except Exception:
            item_name = "找不到商品名稱"
        try:
            item_pics = item.find_elements(By.CSS_SELECTOR,".swiper-wrapper img")
            item_img_url_list = []
            for item_img in item_pics:
                item_img_url = item_img.get_attribute("src")
                item_img_url_list.append(item_img_url)
        except Exception:
            item_img_url_list = "找不到圖片"
        try:
            item_cost = item.find_element(By.CSS_SELECTOR,".sc-KLvxH.gYDCaI > span").text
        except Exception:
            item_cost = item.find_element(By.CSS_SELECTOR,".sc-gKcDdr.sc-jMupca.kwvRcF.gAgQoV").text
        except Exception:
            item_cost = "找不到商品價格"
        try:
            purchase_link = item.get_attribute("href")
        except Exception:
            purchase_link = "找不到購買連結"
            
        item_info = {
            "item_name" : item_name,
            "item_cost" : item_cost,
            "item_img_url_list" : item_img_url_list,
            "purchase_link" : purchase_link
        }
        print(item_info)
get_item_info()

{'item_name': 'Nike 錢包 Icon Air Max 90 Card Wallet 灰 紅 皮革 卡片夾 皮夾 N100974006-8OS', 'item_cost': '$617', 'item_img_url_list': ['https://img.yec.tw/zp/MerchandiseImages/B65B1D831C-SP-16627725.jpg', 'https://img.yec.tw/zp/MerchandiseImages/08D17510FC-SP-16627725.jpg', 'https://img.yec.tw/zp/MerchandiseImages/666E26B82C-SP-16627725.jpg', 'https://img.yec.tw/zp/MerchandiseImages/603B24B3B8-SP-16627725.jpg', 'https://img.yec.tw/zp/MerchandiseImages/DC56A24757-SP-16627725.jpg', 'https://img.yec.tw/zp/MerchandiseImages/62263182B4-SP-16627725.jpg'], 'purchase_link': 'https://tw.buy.yahoo.com/gdsale/Nike-%E9%8C%A2%E5%8C%85-Icon-Air-Max-11028279.html'}
{'item_name': 'Nike 錢包 Icon Air Force 1 Card Wallet 黑 皮革 卡片夾 皮夾 N100973801-3OS', 'item_cost': '$514', 'item_img_url_list': ['https://img.yec.tw/zp/MerchandiseImages/1F1362A26C-SP-16627648.jpg', 'https://img.yec.tw/zp/MerchandiseImages/451ADC9CE1-SP-16627648.jpg', 'https://img.yec.tw/zp/MerchandiseImages/1BAC076633-SP-16627648.jpg', 'https://img.yec.

In [ ]:
pages = 10
for i in range(pages-1):
    page_numbers = driver.find_elements(By.CSS_SELECTOR, ".Pagination__numberContainer___2oWVw a")
    for page_number in page_numbers:
        if page_number.text == str(i+2):
            print(page_number.text)
            page_number.click()
            time.sleep(4)

10
1
2
3
4
5
6
7
8
9
10


In [67]:
# 點下一頁
try:
    driver.find_element(By.CSS_SELECTOR, ".Pagination__icoArrowRight___2KprV").click()
except Exception:
    print("沒有下一頁按鈕")

沒有下一頁按鈕


In [100]:
if not os.path.exists("yahoo購物中心"):
    os.mkdir(f"yahoo購物中心")